# 專題（二）：建置 Bert 新聞觀點分類器之資料集

## 專案目標
- 目標：請試著建置 BertForSequenceClassification 看得懂的兩個句子分類問題的資料集  NewsPairDataset
- 資料集 in archive.zip：
    - 包含：train.csv、test.csv、solution.csv
    - 資料來源：https://www.kaggle.com/wsdmcup/wsdm-fake-news-classification
    - 資料中包含兩個新聞標題 title1_zh 和 title2_zh，並且給予這兩篇新聞的相關性，分別可能是：agreed, unrelated, disagreed

## 實作提示
- STEP1：解壓縮 archive.zip，並且讀取 train.csv 和 test.csv 檔案
- STEP2：繼承 torch.utils.data.Dataset 並實作 NewsPairDataset，其中需要用到 bert tokenizer(請參考官方對 BertForSequenceClassification 的說明)，特別注意兩個句子間必須要有分隔符號 SEP
- STEP3：因為每一個從 NewsPairDataset 來的樣本長度都不一樣，所以需要實作 collate_fn，來zero padding 到同一序列長度
- STEP4：使用 torch.utils.data.DataLoader 來創造 train_loader 和 valid_loader

## 重要知識點：專題結束你可以學會
- 如何讀取並處理 NLP 資料，產生可以適用 BertForSequenceClassification 兩個句子分類問題的資料集
- 了解 BERT 的 2-Sequence Classification 任務如何進行

In [1]:
# from: https://www.kaggle.com/wsdmcup/wsdm-fake-news-classification
!unzip archive.zip

Archive:  archive.zip
  inflating: solution.csv            
  inflating: test.csv                
  inflating: train.csv               


In [2]:
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from transformers import BertTokenizer

In [3]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [4]:
df_train = df_train[['title1_zh', 'title2_zh', 'label']].dropna(axis=0).reset_index(drop=True)
df_test = df_test[['id', 'title1_zh', 'title2_zh']].dropna(axis=0).reset_index(drop=True)

In [5]:
ALL_LABELS = ['agreed', 'unrelated', 'disagreed']

In [6]:
MODEL_NAME = 'bert-base-chinese'

In [7]:
# 建置數據集
class NewsPairDataset(Dataset):
    def __init__(self, tokenizer, df, max_len=512):
        self.tokenizer = tokenizer
        self.df = df
        self.max_len = max_len

    def __getitem__(self, idx):
        text1 = self.df.loc[idx, 'title1_zh']
        text2 = self.df.loc[idx, 'title2_zh']
        label = self.df.loc[idx, 'label'] if 'label' in self.df.columns else None

        text1_tokens = self.tokenizer.tokenize(text1)
        text2_tokens = self.tokenizer.tokenize(text2)
        len_all_tokens = len(text1_tokens) + len(text2_tokens) + 2
        if len_all_tokens > self.max_len:
            limit_num = (self.max_len - 2) // 2
            text1_tokens = text1_tokens[:limit_num]
            text2_tokens = text2_tokens[:limit_num]

        input = {}
        word_pieces = ['[CLS]'] + text1_tokens + ['[SEP]'] + text2_tokens
        input['input_ids'] = torch.LongTensor(self.tokenizer.convert_tokens_to_ids(word_pieces))
        pos_sep = word_pieces.index('[SEP]')
        input['token_type_ids'] = torch.LongTensor(
            [0] * (pos_sep + 1) + [1] * (len(word_pieces) - pos_sep - 1)
        )
        input['attention_mask'] = torch.LongTensor([1] * len(word_pieces))

        if label:
            label = torch.tensor(ALL_LABELS.index(label))

        return input, label

    def __len__(self):
        return len(self.df)

In [8]:
def create_mini_batch(samples):
    input_ids = []
    token_type_ids = []
    attention_mask = []
    labels = []
    for s in samples:
        input_ids.append(s[0]['input_ids'].squeeze(0))
        token_type_ids.append(s[0]['token_type_ids'].squeeze(0))
        attention_mask.append(s[0]['attention_mask'].squeeze(0))
        if s[1]:
            labels.append(s[1])

    # zero pad 到同一序列長度
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=0)
    token_type_ids = pad_sequence(token_type_ids, batch_first=True, padding_value=0)
    attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
 
    if labels:
        labels = torch.stack(labels)
        return input_ids, token_type_ids, attention_mask, labels
    else:
        return input_ids, token_type_ids, attention_mask

In [9]:
train_batch_size = 32
eval_batch_size = 512

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

dataset = NewsPairDataset(tokenizer, df_train)

train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=train_batch_size,
    collate_fn=create_mini_batch,
    shuffle=True
)
valid_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=eval_batch_size,
    collate_fn=create_mini_batch
)